1 MINIST数据集简介，该数据集共有四个文件，即[(训练数据，训练数据标签):60000，(测试数据，测试数据标签):10000]

In [3]:
# some simple sets up

import os
import scipy.misc
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# magic setting
%matplotlib inline
%load_ext autoreload
%autoreload 2

# 忽略警告
import warnings
warnings.filterwarnings('ignore')

# 载入数据
from tensorflow.examples.tutorials.mnist import input_data
minist = input_data.read_data_sets("MINIST_DATA",one_hot=True) # one_hot is True

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Extracting MINIST_DATA\train-images-idx3-ubyte.gz
Extracting MINIST_DATA\train-labels-idx1-ubyte.gz
Extracting MINIST_DATA\t10k-images-idx3-ubyte.gz
Extracting MINIST_DATA\t10k-labels-idx1-ubyte.gz


In [4]:
# train imges
print("traininig data shape:")
print(minist.train.images.shape)
print(minist.train.labels.shape)
print()

# validation images
print("validation data shape:")
print(minist.validation.images.shape)
print(minist.validation.labels.shape)
print()

# test images
print("test data shape:")
print(minist.test.images.shape)
print(minist.test.labels.shape)

traininig data shape:
(55000, 784)
(55000, 10)

validation data shape:
(5000, 784)
(5000, 10)

test data shape:
(10000, 784)
(10000, 10)


原来的图像数据为一维向量，方便计算，方便保存，下面将此一维向量转换为一维矩阵，即还原为图像

知识点：
- python 的文件操作
- scipy.misc转换为图像的功能

In [12]:
# 在代码中打开或创建一个文件夹的方法
save_dir = 'MINIST_DATA/raw/'
if os.path.exists(save_dir) is False:
    os.makedirs(save_dir)

# 保存前20张图片
for i in range(20):
    image_array = minist.train.images[i,:] # 会降维
    image_raw = np.reshape(image_array,(28,28))
    filename = save_dir+"minist_train_image_%d"%i
    scipy.misc.toimage(image_array,cmin=0.0,cmax=1.0).save(filename)

(28, 28)


ValueError: 'arr' does not have a suitable array shape for any mode.

### 1.1.3 图像标签的独热表示  

啥叫独热表示：one-hot representation,一位有效编码，用N维向量来表示N个类别

In [14]:
# 打印前20张image的独热表示，找到他们的类别
for i in range(20):
    print(minist.train.labels[i,:])

[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


## 1.2 利用TensorFlow 识别MNIST

手写识别程序，模型为softmax模型

### 1.2.1 Softmax回归

1. softmax回归的原理

线性的多分类模型，由logistic回归模型转化而来。前者是一个多分类模型，而后者是一个二分类模型。

softmax --> 标签用one-hot进行编码，logistic用0/1进行编码

softmax函数，打分，概率值，

logit == 打分，

tensorflow 中的一些概念

tensor: 表示节点

占位符：tensor,由用户传递给tf,用来存储样本数据和标签

变量: tensor，指在整个计算过程中是可以改变的值



In [16]:
# 使用tensorflow定义一个Softmax模型，实现MNIST数据集的分类

# 创建占位符用来保存训练集数据
X = tf.placeholder(tf.float32,[None,784]) # --> 指定了数据类型，指定了传入数据的shape
y = tf.placeholder(tf.float32,[None,10])

# 创建变量用来保存权值w 和偏置b
W = tf.Variable(tf.zeros([784,10])) # --> 不需要指明数据类型，需要shape以及一个初始值
b = tf.Variable(tf.zeros([10]))

# 建立输入与输出之间的关系
# 输出为y_pre
y_pred = tf.nn.softmax(tf.matmul(X,W)+b) # matmul --> 表示一个矩阵乘法，

Instructions for updating:
Colocations handled automatically by placer.


定义损失函数 loss，即交叉熵函数

In [17]:
# 定义交叉熵损失函数
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y*tf.log(y_pred)))

# y = [0,1,0,0,0,0,0,0], y_pred = [a,b,c,d,e]对应于1的那一项y_pred应该接近于1

In [20]:
# 定义一个优化求解器进行求解
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

# 0.01表示学习率 learning rate

In [21]:
# 创建会话，准备求解该模型了
sess = tf.InteractiveSession()

# 初始化所有变量，分配内存
tf.global_variables_initializer().run()

会话表示节点进行计算的上下文

在会话中初始化变量，在会话中保存变量


In [25]:
# 进行1000步的梯度下降优化
for _ in range(1000):
    # 用minibatch
    batch_x,batch_y = minist.train.next_batch(100)
    sess.run(train_step,feed_dict={X:batch_x,y:batch_y}) # 使用feed_dict喂入数据

# 占位符的值不会被保存，每一次都可以传入不同的值

In [30]:
correct_pred = tf.equal(tf.arg_max(y,1),tf.arg_max(y_pred,1))
# y_pred 已经计算 出来了，
# y 是占位符还要等待着用户输入数据
# 每一次运算都要通过sess.run()进行
# tf.cast() 进行数据间的转换工作

In [31]:
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32))

In [33]:
# 获得最终模型的准确率
print(sess.run(accuracy,feed_dict={X:minist.test.images,y:minist.test.labels}))

0.9024


 **模型的预测准确率**

## 1.2.2 两层卷积网络分类

In [38]:
# 将图像还原为原来的shape
X_image = tf.reshape(X,[-1,28,28,1]) # X --> 占位符

# [N,W,H,D]


In [43]:
# 定义四个函数，用来初始化权重，与偏置，卷积操作以及池化操作

def weight_variable(shape):
    initial = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(initial)

def bia_variable(shape):
    initial = tf.constant(0.1,shape=shape)
    return tf.Variable(initial)

def conv2d(X,W): # X表示输入的，W表示filter
    return tf.nn.conv2d(X,W,strides=[1,1,1,1],padding='SAME')

def max_pool_2x2(X):
    return tf.nn.max_pool(X,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

In [44]:
# 第一层卷积
W_conv1 = weight_variable([5,5,1,32]) # 5X5 表示大小，1表示channel，32表示filter的数量
b_conv1 = bia_variable([32])
h_conv1 = tf.nn.relu(conv2d(X_image,W_conv1)+b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

In [50]:
# 第二层卷积
W_conv2 = weight_variable([5,5,32,64]) # 5X5 32 个通道， 64个filter
b_conv2 = bia_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1,W_conv2)+b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [51]:
# 全连接层
W_fc1 = weight_variable([7*7*64,1024]) # 1024个neuron
b_fcl = bia_variable([1024])
h_pool2_flat = tf.reshape(h_pool2,[-1,7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1)+b_fcl)

In [52]:
# 在该全连接层使用dropout技术
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1,keep_prob)

In [53]:
# 将全连接层转换为最后的输入
W_fc2 = weight_variable([1024,10])
b_fc2 = bia_variable([10])
y_conv = tf.matmul(h_fc1_drop,W_fc2) + b_fc2

In [55]:
# 定义交叉熵损失函数
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

In [56]:
# 定义测试的准确率
correct_pred = tf.equal(tf.argmax(y_conv,1),tf.arg_max(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32))

In [59]:
# 创建session
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
for i in range(20000):
    batch = minist.train.next_batch(50)
    
    # epoch
    if i %100 == 0:
        train_accuracy = accuracy.eval(feed_dict={X:batch[0],y:batch[1],keep_prob:1.0})
        print("step %d, train accuracy %g"%(i,train_accuracy))
    train_step.run(feed_dict={X:batch[0],y:batch[1],keep_prob:0.5})

step 0, train accuracy 0.04
step 100, train accuracy 0.82
step 200, train accuracy 0.86
step 300, train accuracy 0.96
step 400, train accuracy 0.92
step 500, train accuracy 0.96
step 600, train accuracy 0.92
step 700, train accuracy 0.92
step 800, train accuracy 0.92
step 900, train accuracy 0.94
step 1000, train accuracy 0.98
step 1100, train accuracy 0.96
step 1200, train accuracy 0.98
step 1300, train accuracy 0.92
step 1400, train accuracy 0.94
step 1500, train accuracy 0.98
step 1600, train accuracy 0.94
step 1700, train accuracy 0.98
step 1800, train accuracy 0.94
step 1900, train accuracy 0.94
step 2000, train accuracy 1
step 2100, train accuracy 1
step 2200, train accuracy 0.98
step 2300, train accuracy 0.98
step 2400, train accuracy 0.98
step 2500, train accuracy 0.94
step 2600, train accuracy 1
step 2700, train accuracy 1
step 2800, train accuracy 0.98
step 2900, train accuracy 0.98
step 3000, train accuracy 0.96
step 3100, train accuracy 0.96
step 3200, train accuracy 1
step